#Vector de caracteristicas

## Dependencias

In [ ]:
# pyarrow might be needed to read the data
!python -m pip install Cython
!python -m pip install -e git+https://git@github.com/alercebroker/turbo-fats#egg=turbofats
!python -m pip install -e git+https://git@github.com/alercebroker/mhps#egg=mhps
!python -m pip install -e git+https://git@github.com/alercebroker/P4J#egg=P4J
!python -m pip install pyarrow
!python -m pip install -e git+https://git@github.com/alercebroker/lc_classifier#egg=lc_classifier
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining turbofats from git+https://****@github.com/alercebroker/turbo-fats#egg=turbofats
  Cloning https://****@github.com/alercebroker/turbo-fats to ./src/turbofats
  Running command git clone -q 'https://****@github.com/alercebroker/turbo-fats' /content/src/turbofats
  Running setup.py develop for turbofats
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining mhps from git+https://****@github.com/alercebroker/mhps#egg=mhps
  Cloning https://****@github.com/alercebroker/mhps to ./src/mhps
  Running command git clone -q 'https://****@github.com/alercebroker/mhps' /content/src/mhps
  Running setup.py develop for mhps
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining P4J from gi

## Librerias para crear dataframe de curvas de luz


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from lc_classifier.utils import LightcurveBuilder

##Carga de datos en Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Librerias para encontrar el vector de caracteristicas

In [ ]:
from lc_classifier.features import MHPSExtractor, PeriodExtractor, GPDRWExtractor
from lc_classifier.features import FoldedKimExtractor
from lc_classifier.features import HarmonicsExtractor, IQRExtractor
from lc_classifier.features import PowerRateExtractor
from lc_classifier.features import TurboFatsFeatureExtractor

from lc_classifier.features import FeatureExtractorComposer

In [ ]:
bands = ['r','b']
feature_extractor = FeatureExtractorComposer(
    [
        MHPSExtractor(bands),
        PeriodExtractor(bands),
        GPDRWExtractor(bands),
        FoldedKimExtractor(bands),
        HarmonicsExtractor(bands),
        IQRExtractor(bands),
        PowerRateExtractor(bands),
        TurboFatsFeatureExtractor(bands)
    ]
)

## Calculamos el vector de caracteristicas para varias curvas

In [ ]:
ruta="/content/drive/MyDrive/Base_de_datos/RRL/RRab"  #Seleccionamos la carpeta donde estan los archivos .time
lista=[file for file in os.listdir(ruta)]

lightcurves_RRab_4000_7000 = []
for file in lista:
    data = pd.read_csv(f"/content/drive/MyDrive/Base_de_datos/RRL/RRab/{file}", delimiter="\t",header=None)
    data_2 =data.drop([0,1,2,3],axis=0)
    data_3 =data_2[0].str.split(expand=True)
    data_3.columns = ["data", "MagR", "ErMagR","MagB","ErMagB"]
    lightcurve_builder = LightcurveBuilder(f'{file}')
    m=1
    for band in bands:
        n_obs_band = len(data_3["data"].to_numpy())
        time = data_3["data"].to_numpy().astype(np.float)
        magnitude = data_3[data_3.columns[m]].to_numpy().astype(np.float)
        error =data_3[data_3.columns[m]].to_numpy().astype(np.float)
        lightcurve_builder.add_band(
            band,
            time,
            magnitude,
            error)
        m=3

    lightcurve = lightcurve_builder.build_dataframe()
    lightcurves_RRab.append(lightcurve)

lightcurves_RRab= pd.concat(lightcurves_RRab)

In [ ]:
#Extraemos los vectores
features_RRab  = feature_extractor.compute_features(lightcurves_RRab

 /content/src/lc-classifier/lc_classifier/features/extractors/gp_drw_extractor.py:52: RuntimeWarning:overflow encountered in exp
 /usr/local/lib/python3.7/dist-packages/celerite2/numpy.py:108: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.7/dist-packages/celerite2/numpy.py:108: RuntimeWarning:divide by zero encountered in true_divide
 /usr/local/lib/python3.7/dist-packages/celerite2/core.py:427: RuntimeWarning:invalid value encountered in double_scalars
 /content/src/lc-classifier/lc_classifier/features/extractors/gp_drw_extractor.py:71: RuntimeWarning:divide by zero encountered in double_scalars


In [ ]:
#Descargamos un archivo XLSX con los vectores
features_RRab_4000_7000.to_excel("features_RRab_4000_7000.xlsx")